<h2>Duplicate Job Listings</h2>
<p>Difficulty: Easy</p>
<i><b>Question: </b>Assume you're given a table containing job postings from various companies on the LinkedIn platform.
 Write a query to retrieve the count of companies that have posted duplicate job listings.</i>

[Link to the question](https://datalemur.com/questions/duplicate-job-listings)

In [ ]:
SELECT 
    COUNT(*)
FROM (
    SELECT company_id
    FROM job_listings
    GROUP BY company_id
    HAVING count(company_id) > 1
    ) as filter_table